## Homework 08 - Deep Learning

### Import

In [ ]:
# import libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Tensorflow libraries
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
# setting seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Create an instance of ImageDataGenerator with a preprocessing function
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
# Create a data generator for traning images
train_ds = train_gen.flow_from_directory(
    # Path to the directory with the training images
    'data/raw/data/train', 
    # Resize all images to 150x150
    target_size=(200,200), 
    # Number of images to be yielded in each batch
    batch_size=20,
    class_mode='categorical',
    shuffle = True,
    )

Found 800 images belonging to 2 classes.


### Loading the images


In [5]:
# Create an instance of ImageDataGenerator with a preprocessing function
train_gen = ImageDataGenerator(rescale=1./255)

In [6]:
# Get the next batch of images and labels from the generator
X, y = next(train_ds)

In [7]:
# create a data generator for validation images with prepocessing
val_gen = ImageDataGenerator(rescale=1./255)

# Create a data generator for validation images
val_ds = val_gen.flow_from_directory(
    'data/raw/data/test', 
    target_size=(200,200), 
    batch_size=20,
    class_mode='categorical',
    shuffle=True
    )

Found 201 images belonging to 2 classes.


In [8]:
from tensorflow.keras import layers, models, optimizers

# Function to define model by adding new dense layer and dropout
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                          include_top=False,
                          input_shape=(200,200,3))

    base_model.trainable = False
    
    #########################################
    
    inputs = tf.keras.Input(shape=(200,200,3))
    base = base_model(inputs, training=False)
    vectors = tf.keras.layers.GlobalAveragePooling2D()(base)
    inner = tf.keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = tf.keras.layers.Dropout(droprate)(inner) # add dropout layer
    outputs = tf.keras.layers.Dense(2)(drop)  # Change the number of output units to 2
    model = tf.keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

    def make_model(learning_rate=0.002, momentum=0.8):
        model = models.Sequential()
        model.add(layers.Input(shape=(200, 200, 3)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        optimizer = optimizers.SGD(lr=learning_rate, momentum=momentum)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [9]:
model = make_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │       204,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,066,582 (80.36 MB)

 Trainable params: 205,102 (801.18 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [10]:
# Train the model and store the history
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

# Calculate the median of training accuracy
training_accuracies = history.history['accuracy']
median_training_accuracy = np.median(training_accuracies)
print(f"Median Training Accuracy: {median_training_accuracy}")

Epoch 1/10


/Users/zaccaria/.pyenv/versions/3.11.7/envs/class_img/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 634ms/step - accuracy: 0.7545 - loss: 1.0947 - val_accuracy: 0.9403 - val_loss: 0.1390
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 620ms/step - accuracy: 0.9082 - loss: 0.2439 - val_accuracy: 0.9652 - val_loss: 0.1301
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 621ms/step - accuracy: 0.9554 - loss: 0.1359 - val_accuracy: 0.9602 - val_loss: 0.1087
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 620ms/step - accuracy: 0.9481 - loss: 0.1559 - val_accuracy: 0.9652 - val_loss: 0.0898
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 621ms/step - accuracy: 0.9714 - loss: 0.0892 - val_accuracy: 0.9453 - val_loss: 0.1225
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 622ms/step - accuracy: 0.9681 - loss: 0.0703 - val_accuracy: 0.9602 - val_loss: 0.0936
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 620ms/step - accuracy: 0.9723 - loss: 0.0836 - val_accuracy: 0.9602 - val_loss: 0.1252
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 621ms/step - accuracy: 0.9883 - loss: 0.0440 - val_accuracy: 0.960

In [11]:
# Extract the training losses from the history object
training_losses = history.history['loss']

# Calculate the standard deviation of the training losses
std_training_loss = np.std(training_losses)
print(f"Standard Deviation of Training Loss: {std_training_loss:.3f}")

Standard Deviation of Training Loss: 0.188


In [12]:
# Create an instance of ImageDataGenerator with the specified augmentations
train_gen_augmented = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a data generator for training images with augmentations
train_ds_augmented = train_gen_augmented.flow_from_directory(
    'data/raw/data/train', 
    target_size=(200,200), 
    batch_size=20,
    class_mode='categorical',
    shuffle=True
)

Found 800 images belonging to 2 classes.


In [13]:
from tensorflow.keras import layers, models, optimizers

# Function to define model by adding new dense layer and dropout
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(weights='imagenet',
                          include_top=False,
                          input_shape=(200,200,3))

    base_model.trainable = False
    
    #########################################
    
    inputs = tf.keras.Input(shape=(200,200,3))
    base = base_model(inputs, training=False)
    vectors = tf.keras.layers.GlobalAveragePooling2D()(base)
    inner = tf.keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = tf.keras.layers.Dropout(droprate)(inner) # add dropout layer
    outputs = tf.keras.layers.Dense(2)(drop)  # Change the number of output units to 2
    model = tf.keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

    def make_model(learning_rate=0.002, momentum=0.8):
        model = models.Sequential()
        model.add(layers.Input(shape=(200, 200, 3)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        optimizer = optimizers.SGD(lr=learning_rate, momentum=momentum)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [15]:
# Train the model and store the history
history = model.fit(
    train_ds_augmented,
    epochs=10,
    validation_data=val_ds
)

# Calculate the median of training accuracy
training_accuracies = history.history['accuracy']
average_training_accuracy = np.average(training_accuracies)
print(f"Median Training Accuracy: {average_training_accuracy}")

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 26s 643ms/step - accuracy: 0.9361 - loss: 0.1619 - val_accuracy: 0.9204 - val_loss: 0.1919
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 631ms/step - accuracy: 0.9271 - loss: 0.2068 - val_accuracy: 0.9552 - val_loss: 0.1307
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 630ms/step - accuracy: 0.9375 - loss: 0.1902 - val_accuracy: 0.9254 - val_loss: 0.1492
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 631ms/step - accuracy: 0.9120 - loss: 0.2387 - val_accuracy: 0.9303 - val_loss: 0.1872
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 630ms/step - accuracy: 0.9121 - loss: 0.2084 - val_accuracy: 0.9303 - val_loss: 0.1553
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 632ms/step - accuracy: 0.9428 - loss: 0.2216 - val_accuracy: 0.9403 - val_loss: 0.1756
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 632ms/step - accuracy: 0.9397 - loss: 0.2263 - val_accuracy: 0.9602 - val_loss: 0.1152
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 25s 631ms/step - accuracy: 0.9322 - loss: 0.1541 - val_accu

In [16]:
# Extract the validation accuracies from the history object
val_accuracies = history.history['val_accuracy']

# Calculate the average of the last 5 epochs
average_val_accuracy = np.mean(val_accuracies[-5:])
print(f"Average Test Accuracy for the last 5 epochs: {average_val_accuracy:.2f}")

Average Test Accuracy for the last 5 epochs: 0.93


In [17]:

# Extract the validation losses from the history object
val_losses = history.history['val_loss']

# Calculate the mean of the validation losses
mean_val_loss = np.mean(val_losses)
print(f"Mean Test Loss for all epochs: {mean_val_loss:.2f}")

Mean Test Loss for all epochs: 0.17
